<a target="_blank" href="https://colab.research.google.com/github/sonder-art/automl_p24/blob/main/codigo/ml_practico/01_default_etl_train_test.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Prediccion de Default en Prestamos

Para este proyecto utilizaremos un sample de los datos de [Lending Club](https://www.kaggle.com/wordsforthewise/lending-club). La idea es predecir si cierto usuario cometera Default basado en informacion que la plataforma recolecta. Esto nos ayudara a mejorar la metodologia/pipeline de prestamo.  



In [1]:


# Load libraries
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import (train_test_split, KFold, cross_val_score, 
GridSearchCV, StratifiedKFold)



# Dataset

Contiene los prestamos de esta plataforma:
+ periodo 2007-2017Q3.
+ 887mil observaciones, sample de 100mil 
+ 150 variables
+ Target: loan status

**En estricto sentido hay un error al no separar el train y el test al inicio, pues tomamos informacion con lo que seria nuestro test. Sin embargo, dado que utilizamos 100K de los mas de 800K datos disponibles podemos verlo como si todo fuera train.  
Cuando realicen sus proyectos, separen test y train, y vayan guardando todas las transformaciones que van realiziando al train, para luego aplicarlas al test!**

# ETL

In [5]:
# Por si se necesita pyarrow
#!pip install pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 5.1 MB/s eta 0:00:0000:0100:01


In [6]:
if os.path.isfile('loans_sampe_data.parquet'):
    loans = pd.read_parquet('loans_sampe_data.parquet')
else:
    print('Descargando Datos')
    loans = pd.read_parquet('https://github.com/sonder-art/automl_p24/raw/main/codigo/ml_practico/loans_sampe_data.parquet')
    loans.to_parquet('loans_sampe_data.parquet')

In [7]:
loans.shape

(100000, 151)

Revisemos la columna del Warning, parece que es texto, que describe el objetivo del prestamo.

In [8]:
print(loans.columns[19])
loans.loc[:,loans.columns[19]].unique()

desc


array([None, ' ',
       'Looking to borrow to purchase a used boat we are interested in',
       'I support myself and my mother and have taken on debt to assist with her bills and health care needs. I would like to pay down debt faster through refinancing at a lower rate.',
       "Trying to pay a friend back for apartment broker's fee incurred from as well as credit card stuff."],
      dtype=object)

In [6]:
loans.shape

(100000, 151)

In [10]:
loans

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,0,38098114,NaN,15000.0,15000.0,15000.0,60 months,12.39,336.64,C,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
1,1,36805548,NaN,10400.0,10400.0,10400.0,36 months,6.99,321.08,A,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
2,2,37842129,NaN,21425.0,21425.0,21425.0,60 months,15.59,516.36,D,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
3,3,37612354,NaN,12800.0,12800.0,12800.0,60 months,17.14,319.08,D,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
4,4,37662224,NaN,7650.0,7650.0,7650.0,36 months,13.66,260.20,C,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,22454240,NaN,8400.0,8400.0,8400.0,36 months,9.17,267.79,B,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
99996,99996,11396920,NaN,10000.0,10000.0,10000.0,36 months,12.99,336.90,C,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
99997,99997,8556176,NaN,30000.0,30000.0,30000.0,60 months,20.99,811.44,E,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
99998,99998,24023408,NaN,8475.0,8475.0,8475.0,36 months,24.99,336.92,F,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN


In [11]:
loans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Columns: 151 entries, Unnamed: 0 to settlement_term
dtypes: float64(115), int64(2), object(34)
memory usage: 115.2+ MB


La primera columna parece ser un indice, podemos eliminarla.

In [12]:
type(np.nan)

float

In [13]:
dataset = loans
dataset.head(10)

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,0,38098114,NaN,15000.0,15000.0,15000.0,60 months,12.39,336.64,C,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
1,1,36805548,NaN,10400.0,10400.0,10400.0,36 months,6.99,321.08,A,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
2,2,37842129,NaN,21425.0,21425.0,21425.0,60 months,15.59,516.36,D,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
3,3,37612354,NaN,12800.0,12800.0,12800.0,60 months,17.14,319.08,D,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
4,4,37662224,NaN,7650.0,7650.0,7650.0,36 months,13.66,260.20,C,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
5,5,37822187,NaN,9600.0,9600.0,9600.0,36 months,13.66,326.53,C,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
6,6,37741884,NaN,2500.0,2500.0,2500.0,36 months,11.99,83.03,B,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
7,7,37854444,NaN,16000.0,16000.0,16000.0,60 months,11.44,351.40,B,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
8,8,36804663,NaN,23325.0,23325.0,23325.0,36 months,14.31,800.71,C,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
9,9,37642222,NaN,5250.0,5250.0,5250.0,36 months,11.44,172.98,B,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN


In [18]:
dataset['Unnamed: 0'].unique().shape

(100000,)

In [19]:
del dataset['Unnamed: 0']

Diccionario con la descripcion de los datos.

In [22]:
datos_dict = pd.read_excel('https://resources.lendingclub.com/LCDataDictionary.xlsx')
datos_dict.columns = ['feature', 'description']

In [13]:
datos_dict

,feature,description
0,acc_now_delinq,The number of accounts on which the borrower i...
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan...
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by th...
...,...,...
148,settlement_amount,The loan amount that the borrower has agreed t...
149,settlement_percentage,The settlement amount as a percentage of the p...
150,settlement_term,The number of months that the borrower will be...
151,NaN,NaN


# EDA

## Variable Target -> Default de Prestamo

In [23]:
dataset['loan_status'].value_counts(dropna=False)

loan_status
Fully Paid            69982
Charged Off           16156
Current               12726
Late (31-120 days)      730
In Grace Period         264
Late (16-30 days)       139
Default                   3
Name: count, dtype: int64

En este caso las variables que nos interesan son los prestamos terminados, ya sea porque pagaron `Fully Paid` o porque no pagaron y el prestamo se dio por perdido `Charged Off`.  
En este caso las demas opciones, incluyendo `Default`, implican que el prestamo sigue en curso y no ha sido concluido, asi que las ignoraremos.

In [15]:
dataset = dataset.loc[dataset['loan_status'].isin(['Fully Paid', 
                                                   'Charged Off'])]
dataset['loan_status'].value_counts(normalize=True, dropna=False)

loan_status
Fully Paid     0.812441
Charged Off    0.187559
Name: proportion, dtype: float64

Podemos observar que tenemos una distribucion de labels cercana al 80/20. 81% de los prestamos fueron liquidados exitosamente, mientras que el 19% nunca fue pagado.  

Creemos el label de 0 o 1 en la columna `charged_off`.

In [24]:
dataset['charged_off'] = (dataset['loan_status'] == 'Charged Off').apply(np.uint8)
dataset.drop('loan_status', axis=1, inplace=True)
dataset['charged_off'].unique()

array([0, 1], dtype=uint8)

## Separacion Train y Validacion 


En este caso como existen datos que no usamos, usaremos la validacion como test.

In [26]:
# Estratificamos de manera que esten balanceados los datos de acuerdo a 
# nuestro label 'charged_off'
data_train, data_test = train_test_split(dataset,
                                    stratify=dataset['charged_off'], 
                                    test_size=0.2, random_state=453890)

In [28]:
print(data_train.shape)
data_train['charged_off'].value_counts(normalize=True, dropna=False)

(80000, 150)


charged_off
0    0.838437
1    0.161562
Name: proportion, dtype: float64

In [29]:
print(data_test.shape)
data_test['charged_off'].value_counts(normalize=True, dropna=False)


(20000, 150)


charged_off
0    0.83845
1    0.16155
Name: proportion, dtype: float64

**Recuerda evitar decisiones utilizando el Test** Aunque para hacer los modelos compatibles hay que aplicar exactamente las mismas transformaciones que apliques al train.

In [30]:
data_train.to_parquet('data_train.parquet')
data_test.to_parquet('data_test.parquet')